In [1]:
#setting for Django project
import os
import django
# Set the Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'deliveryoptimizer.settings')

# Initialize Django
django.setup()

### Transform container as per Container model

In [2]:
import json
from pathlib import Path

# Define the base directory and file paths

base_dir_data = Path("./cargo_storageOpt/CLP_GA/CLP_BR_Data")
input_file_path = base_dir_data / "containers.json"
output_file_path = base_dir_data / "transformed_containers.json"

# Ensure the input file exists
if not input_file_path.exists():
    raise FileNotFoundError(f"Input file not found: {input_file_path}")

# Ensure the output directory exists
output_file_path.parent.mkdir(parents=True, exist_ok=True)
def transform_containers(file_path):
    OPENING_SHORTCUTS = {
        "Open to Top": "T",
        "Open to Back": "B",
        "Open to Side": "S"
    }
    
    with open(file_path, "r") as file:
        data = json.load(file)
    
    transformed = []
    containers = data.get("containers", [])
    for idx, container in enumerate(containers):
        transformed.append({
            "cont_ID": container["name"],
            "external_length": container["external_dimention"]["m"][0],
            "external_width": container["external_dimention"]["m"][1],
            "external_height": container["external_dimention"]["m"][2],
            "internal_length": container["internal_dimention"]["m"][0],
            "internal_width": container["internal_dimention"]["m"][1],
            "internal_height": container["internal_dimention"]["m"][2],
            "opening_type": OPENING_SHORTCUTS[container["container_opening"]],
            "payload": container["payload"],
            "tare_weight": container["tare_weight"],
            "sort_id": idx + 1
        })
    return transformed

# Transform the data and save to a new file
transformed_data = transform_containers(input_file_path)

with open(output_file_path, "w") as file:
    json.dump(transformed_data, file, indent=4)

print(f"Transformed data saved to {output_file_path}")


Transformed data saved to cargo_storageOpt\CLP_GA\CLP_BR_Data\transformed_containers.json


### Save transformed data to containers DB
Please check DB update through POST req from postman

In [54]:
from cargo_storageOpt.models.container import Container

def add_containers(data):
    containers = []
    errors = []
    
    required_fields = [
        'opening_type', 'cont_ID', 'sort_id', 'tare_weight', 'payload',
        'external_length', 'external_width', 'external_height',
        'internal_length', 'internal_width', 'internal_height'
    ]

    for idx, container_data in enumerate(data):
        missing_fields = [field for field in required_fields if field not in container_data]
        if missing_fields:
            errors.append({
                'index': idx,
                'error': f'Missing fields: {", ".join(missing_fields)}'
            })
            continue
        
        try:
            # Create container object
            container = Container(
                opening_type=container_data['opening_type'],
                cont_ID=container_data['cont_ID'],
                sort_id=container_data['sort_id'],
                tare_weight=container_data['tare_weight'],
                payload=container_data['payload'],
                external_length=container_data['external_length'],
                external_width=container_data['external_width'],
                external_height=container_data['external_height'],
                internal_length=container_data['internal_length'],
                internal_width=container_data['internal_width'],
                internal_height=container_data['internal_height']
            )
            
            # Save the container to the database
            container.save()
        
        except Exception as e:
            errors.append({'index': idx, 'error': str(e)})

    return errors  # Return any errors if present


In [55]:
import threading

# Ensure the function 'add_containers' accepts a parameter, like 'data'
t = threading.Thread(target=add_containers, args=(transformed_data,))
t.start()

### Update orders

In [66]:
from cargo_storageOpt.models.UpdateOrders import reset_order_fetched_for_range

t = threading.Thread(target=reset_order_fetched_for_range, args=(4,6))
t.start()

Updated 3 orders with order_fetched set to False and supportRatio set to 0.85.
ID: 4, order_fetched: False, supportRatio: 0.85
ID: 5, order_fetched: False, supportRatio: 0.85
ID: 6, order_fetched: False, supportRatio: 0.85
